In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, LeakyReLU, Embedding
from keras.optimizers import Adam, RMSprop
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook, tqdm
from numpy import zeros, ones
from numpy.random import randn, randint
import os

In [3]:
path="/content/drive/MyDrive/paper/env_conditions/data/prepared_data/all_conditions.csv"
df = pd.read_csv(path)
df = df.drop(['Unnamed: 0'], axis=1)
df.isna().sum()
df.head(1)

,Stn Id,Stn Name,CIMIS Region,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F),Target
0,2,FivePoints,San Joaquin Valley,1/1/2018,0.06,0.0,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,78.3,51.1,0


# prepare data

In [7]:
df_date = df['Date'].values
day, month, year = [], [], []
for i in df_date:
    data = i.split('/')
    day.append(int(data[1]))
    month.append(int(data[0]))
    year.append(int(data[2]))

temp = pd.DataFrame({'year': year,'month': month,'day': day})
#temp = pd.to_datetime(temp)
#df['Date'] = temp

In [8]:
df1 = df.drop(['Stn Id', 'Stn Name', 'CIMIS Region', 'Date'], axis=1)

In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(['Target'], axis=1)
Y = df['Target']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

Y_train.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)

In [10]:
# from sklearn.preprocessing import LabelEncoder
# category_cols = ['Stn Name', 'CIMIS Region']
# temp1 = X_train.get(category_cols[0])
# temp2 = X_train.get(category_cols[1])


# encoder1 = LabelEncoder()
# encoder1.fit(temp1)
# temp1 = encoder1.transform(temp1)

# encoder2 = LabelEncoder()
# encoder2.fit(temp2)
# temp2 = encoder2.transform(temp2)

# X_train1 = X_train
# X_train1[category_cols[0]] = temp1
# X_train1[category_cols[1]] = temp2

In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

X_train1, Y_train1 = X_train, Y_train

encoder_x = MinMaxScaler()
encoder_x.fit(X_train1)

X_train1 = encoder_x.transform(X_train1)

encoder_y = LabelEncoder()
encoder_y.fit(Y_train1)

Y_train1 = encoder_y.transform(Y_train1)

ValueError: could not convert string to float: 'Pleasanton'

# Build model

In [ ]:
from tqdm import tqdm_notebook
import matplotlib
import matplotlib.pylab as plt
from math import ceil
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input, ReLU, LeakyReLU, Dense, Conv2D, Conv2DTranspose, Concatenate, Dropout
from keras.optimizers import SGD, Adam
from keras.initializers import RandomNormal

In [ ]:
def build_generator(n_classes, z_dim = 100):
    output_g = len(X_train1[0])

    #block1
    in_label = Input(shape=(1,))
    #in_label = Embedding(n_classes, 50)(in_label)
    blockA1 = Dense(128, activation=LeakyReLU(alpha=0.2))(in_label)
    blockA1 = Dense(64, activation=LeakyReLU(alpha=0.2))(blockA1)
    blockA1 = Dense(32, activation=LeakyReLU(alpha=0.2))(blockA1)

    #block2
    in_data = Input(shape=(z_dim,))
    blockA2 = Dense(1024, activation=LeakyReLU(alpha=0.2))(in_data)
    blockA2 = Dense(512, activation=LeakyReLU(alpha=0.2))(blockA2)
    blockA2 = Dense(224, activation=LeakyReLU(alpha=0.2))(blockA2)

    #Concat
    merge = Concatenate()([blockA2, blockA1])

    #main
    g = Dense(128, activation=LeakyReLU(alpha=0.2))(merge)
    g = Dense(63, activation=LeakyReLU(alpha=0.2))(g)
    g = Dense(32, activation=LeakyReLU(alpha=0.2))(g)
    output = Dense(output_g, activation='sigmoid')(g)

    #Define model
    model = Model([in_data, in_label], output)


    print('Build Generator')
    print(model.summary())

    return model

def build_discriminator(n_classes):
    input_d = len(X_train1[0])

    #block1
    in_label = Input(shape=(1,))
    #in_label = Embedding(n_classes, 50)(in_label)
    blockA1 = Dense(64, activation=LeakyReLU(alpha=0.2))(in_label)
    blockA1 = Dense(32, activation=LeakyReLU(alpha=0.2))(blockA1)
    blockA1 = Dense(16, activation=LeakyReLU(alpha=0.2))(blockA1)

    #block2
    in_data = Input(shape=(input_d,))
    blockA2 = Dense(256, activation=LeakyReLU(alpha=0.2))(in_data)
    blockA2 = Dropout(0.5)(blockA2)
    blockA2 = Dense(128, activation=LeakyReLU(alpha=0.2))(blockA2)
    blockA2 = Dropout(0.5)(blockA2)
    blockA2 = Dense(48, activation=LeakyReLU(alpha=0.2))(blockA2)
    blockA2 = Dropout(0.4)(blockA2)

    #Concat
    merge = Concatenate()([blockA2, blockA1])

    #main
    d = Dense(16, activation=LeakyReLU(alpha=0.2))(merge)
    d = Dropout(0.3)(d)
    output = Dense(1, activation='sigmoid')(d)

    #define model
    model = Model([in_data, in_label], output)
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

    print('Build discriminator')
    print(model.summary())

    return model


def build_gan(g_model, d_model):
    #Not train d_model
    d_model.trainable = False

    gen_noise, gen_label = g_model.input
    gen_output = g_model.output
    gan_output = d_model([gen_output, gen_label])
    model = Model([gen_noise, gen_label], gan_output)
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

    print('Build GAN')
    print(model.summary())

    return model


def load_real_samples():
	return [X_train1, Y_train1]


def generate_real_samples(dataset, n_samples):
	# split into images and labels
	data, labels = dataset
	# choose random instances
	ix = randint(0, data.shape[0], n_samples)
	# select images and labels
	X, labels = data[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y


def generate_latent_points(latent_dim, n_samples, n_classes=2):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]


def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    data = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [data, labels_input], y


# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim=100, n_epochs=100, n_batch=512):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
      # enumerate batches over the training set
      for j in range(bat_per_epo):
          # get randomly selected 'real' samples
          [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)

          d_model.trainable = True
          # update discriminator model weights
          d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
          # generate 'fake' examples
          [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
          # update discriminator model weights
          d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)

          d_model.trainable = False
          # prepare points in latent space as input for the generator
          [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
          # create inverted labels for the fake samples
          y_gan = ones((n_batch, 1))
          # update the generator via the discriminator's error
          g_loss, acc = gan_model.train_on_batch([z_input, labels_input], y_gan)
          # summarize loss on this batch
          print(f"G acc: {acc}\nG loss: {g_loss}\nD loss 1: {d_loss1}\nD loss 2: {d_loss2}")
          print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
              (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))

      js = open('/content/drive/MyDrive/paper/env_conditions/Model/CGAN/gan_20240715.log', 'a')
      js.write(f'-----------------------------------------------Epoch {e}-----------------------------------------------\n')
      js.write(f'd loss: {d_loss}\t\t\td acc: {d_acc}\n')
      js.write(f'g loss: {g_loss}\t\t\tg acc: {g_acc}\n')
      js.write('\n\n')
      js.close()
            # save the generator model
      g_model.save(f'/content/drive/MyDrive/paper/env_conditions/Model/CGAN/cgan_generator_{i}.h5')

In [ ]:
# size of the latent space
latent_dim = 100
n_classes = 2
# create the discriminator
d_model = build_discriminator(n_classes)
# create the generator
g_model = build_generator(n_classes,latent_dim)
# create the gan
gan_model = build_gan(g_model, d_model)
# load data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)